# Analyzing PPP Loans Data for Bay Area

*Based on ZIP Codes*

In [49]:
import pandas as pd
import numpy as np
import re
import warnings

warnings.filterwarnings('ignore')

## Part 1: Load Other Tables for PPP Loan Data

The following datasets are external to the PPP Loan Datasets and will be used to match and filter some columns in the PPP Loan files.

`NAICS_codes`: To merge with `PPP_combined` on 6-digit NAICS code, need to match with 4-digit code

`bay_area_zip`: New dataset with zip codes of all of Bay Area (might need to filter for 6 cities)

In [15]:
#Import NAICS 6 and 4 digit concordance table
NAICS_codes = pd.read_csv('naics_data_rsei_v238.csv')
NAICS_codes['2017NAICSCode'] = NAICS_codes['2017NAICSCode'].apply(str)
NAICS_codes.head()

,2017NAICSCode,LongName,Changed2017,TRIIndustrySector,IndustrySubsector,4DigitNAICS,NewIndustry
0,111110,Soybean Farming,NaN,999 Other,1111 Oilseed and Grain Farming,1111,False
1,111120,Oilseed (except Soybean) Farming,NaN,999 Other,1111 Oilseed and Grain Farming,1111,False
2,111130,Dry Pea and Bean Farming,NaN,999 Other,1111 Oilseed and Grain Farming,1111,False
3,111140,Wheat Farming,NaN,999 Other,1111 Oilseed and Grain Farming,1111,False
4,111150,Corn Farming,NaN,999 Other,1111 Oilseed and Grain Farming,1111,False


In [16]:
#Import Bay Area Zip Codes (New Dataset as of 5/28/2021) to filter from PPP Loan data
bay_area_zip = pd.read_csv('cb_2018_us_zcta510_500k_BAY.csv')
bay_area_zip = bay_area_zip.rename(columns={'ZCTA5CE10':'zip_code_5'})
bay_area_zip['zip_code_5'] =bay_area_zip['zip_code_5'].apply(str)
bay_area_zip.head()

,zip_code_5,AFFGEOID10,GEOID10,ALAND10,AWATER10
0,94952,8600000US94952,94952,466821663,2998248
1,94127,8600000US94127,94127,4585722,9359
2,95363,8600000US95363,95363,305162606,1236641
3,95441,8600000US95441,95441,263937111,4704473
4,94574,8600000US94574,94574,330686479,5409337


## Part 2: Sample Analysis on one CSV file

Used `public_up_to_150k_1.csv` to:
1. Analyze NaN values in PPP Loans data. *(See Questions and Notes)
2. Split full dataframe in 2: NaN Zip Codes & Non-NaN Zip Codes
3. Added modified zip code column to Non-NULL dataframe
4. Extracted only Bay Area Zip Codes using table in Part 1

**TODO:** 
- Analyze NaN Zips and add Bay Area & Minority-Owned ones to `bay_area_ppp`
- Merge the NAICS 4-digit codes
- Apply analysis to all files to make `PPP_combined`

In [3]:
# Read in one file
ppp_sub_150_1 = pd.read_csv('public_up_to_150k_1.csv')
ppp_sub_150_1.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
0,5375617707,05/01/2020,101.0,PPP,NOT AVAILABLE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9551.0,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,NaN
1,9677497701,05/01/2020,464.0,PPP,NORTH CHARLESTON HOSPITALITY GROUP LLC,192 College Park Rd,Ladson,NaN,29456-3517,NaN,...,NaN,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN
2,9547167709,05/01/2020,464.0,PPP,Q AND J SERVICES LLC,301 Old Georgetown Road,Manning,NaN,29102-2734,04/20/2021,...,NaN,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN
3,6448037706,05/01/2020,515.0,PPP,OPTIMIZED PROCESS SOLUTIONS DBA AAA INDUSTRIES,24500 CAPITOL,REDFORD,NaN,48239-2446,04/16/2021,...,NaN,NaN,Limited Liability Company(LLC),9551.0,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN
4,9609017706,05/01/2020,464.0,PPP,"D2 POWER SPORTS, LLC",125 Simuel Dr.,Spartanburg,NaN,29303-2085,NaN,...,NaN,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN


### Part 2a. Analyze NaN Zip Codes

Looking Into Businesses with little to no information on location

**Questions:**

1. What should we do with NaN zip codes? What if it is minority-owned but area is unknown?

**Notes:**

There are 124 NaN Zip Codes in `ppp_sub_150_1`. Should we drop them all? Or manually filter them?

Idea - Automate Process:

1. In each table, get all NaN Zip Codes -> New DF
2. In new DF, get Race and Ethnicity where it is answered
3. Filter for anything !White and !Not Hispanic or Latino
4. View DF with indices of minority businesses with no Zip Codes -> Manually Check?
5. If 'BorrowerState' or 'ProjectState' == CA: keep, else: drop from table
6. Manually check remaining rows.

In [19]:
# NaN Example: No location information, Nothing on Race and Ethincity
ppp_sub_150_1.loc[0]

LoanNumber                                                5375617707
DateApproved                                              05/01/2020
SBAOfficeCode                                                    101
ProcessingMethod                                                 PPP
BorrowerName                                           NOT AVAILABLE
BorrowerAddress                                                  NaN
BorrowerCity                                                     NaN
BorrowerState                                                    NaN
BorrowerZip                                                      NaN
LoanStatusDate                                                   NaN
LoanStatus                                               Exemption 4
Term                                                              24
SBAGuarantyPercentage                                            100
InitialApprovalAmount                                         148440
CurrentApprovalAmount             

In [17]:
# Count NaN Zip Codes (BorrowerZip) in table
ppp_sub_150_1['BorrowerZip'].isna().sum()

124

In [28]:
# Get rows where BorrowerZip is NaN
nan_zip_df = ppp_sub_150_1[ppp_sub_150_1['BorrowerZip'].isnull()]
nan_zip_df.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
0,5375617707,05/01/2020,101.0,PPP,NOT AVAILABLE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9551.0,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,NaN
5,9789867710,05/01/2020,101.0,PPP,VULCAN MACHINERY CORPORATION,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Corporation,57328.0,The Huntington National Bank,COLUMBUS,OH,Male Owned,Non-Veteran,NaN
6,9589997709,05/01/2020,101.0,PPP,"TJK KITCHENS &AMP; BREWPUBS, LLC",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Limited Liability Company(LLC),57328.0,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN
8,9662387700,05/01/2020,101.0,PPP,RON GOLDSTONE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,57328.0,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN
9,2767027201,04/16/2020,NaN,PPP,Exemption 6,NaN,NaN,NaN,NaN,02/18/2021,...,0.0,0.0,Subchapter S Corporation,NaN,NaN,NaN,NaN,Unanswered,Unanswered,NaN


In [31]:
# Get Race of these NaN rows
nan_zip_df[nan_zip_df['Race'] != 'Unanswered']['Race']

11                            White
15                            White
22                            White
25                            White
26                            White
48                            White
51                            White
67        Black or African American
76                            White
79                            White
83                            White
87                            White
95                            White
102                           White
103                           White
119                           White
120                           White
130                           White
466347                        Asian
Name: Race, dtype: object

In [32]:
# Get Ethnicity of these NaN rows
nan_zip_df[nan_zip_df['Ethnicity'] != 'Unknown/NotStated']['Ethnicity']

11        Not Hispanic or Latino
22        Not Hispanic or Latino
25        Not Hispanic or Latino
26        Not Hispanic or Latino
48        Not Hispanic or Latino
51        Not Hispanic or Latino
67        Not Hispanic or Latino
76        Not Hispanic or Latino
79        Not Hispanic or Latino
83        Not Hispanic or Latino
87        Not Hispanic or Latino
95        Not Hispanic or Latino
102       Not Hispanic or Latino
103       Not Hispanic or Latino
119       Not Hispanic or Latino
120       Not Hispanic or Latino
130       Not Hispanic or Latino
133           Hispanic or Latino
466347    Not Hispanic or Latino
Name: Ethnicity, dtype: object

***Examples of NaN, Minority-owned businesses***

In [39]:
'''
BorrowerName                                 SUNKISSBIZ LLC
BorrowerAddress                                         NaN
BorrowerCity                                            NaN
BorrowerState                                           NaN
BorrowerZip                                             NaN
...
NAICSCode                                            611610
Race                              Black or African American
Ethnicity                            Not Hispanic or Latino
'''
nan_zip_df.loc[67] #Located in MI -> Drop

In [40]:
'''
BorrowerName                                       ART N FUN STUDIO, INC
BorrowerAddress                                                      NaN
BorrowerCity                                                         NaN
BorrowerState                                                         CA
BorrowerZip                                                          NaN
...
NAICSCode                                                         712110
Race                                                               Asian
Ethnicity                                         Not Hispanic or Latino
'''
nan_zip_df.loc[466347] # Located In Santa Clara, CA, 95051 -> Keep?

In [41]:
'''
BorrowerName                           Exemption 6
BorrowerAddress                                NaN
BorrowerCity                                   NaN
BorrowerState                                  NaN
BorrowerZip                                    NaN
...
NAICSCode                                   561720
Race                                    Unanswered
Ethnicity                       Hispanic or Latino
'''
nan_zip_df.loc[133] # No location information

### Part 2b. Split Dataframe - Get Non-NaN Dataframe 


In [88]:
not_nan_df = ppp_sub_150_1[ppp_sub_150_1['BorrowerZip'].notnull()]

not_nan_df.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
1,9677497701,05/01/2020,464.0,PPP,NORTH CHARLESTON HOSPITALITY GROUP LLC,192 College Park Rd,Ladson,NaN,29456-3517,NaN,...,NaN,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN
2,9547167709,05/01/2020,464.0,PPP,Q AND J SERVICES LLC,301 Old Georgetown Road,Manning,NaN,29102-2734,04/20/2021,...,NaN,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN
3,6448037706,05/01/2020,515.0,PPP,OPTIMIZED PROCESS SOLUTIONS DBA AAA INDUSTRIES,24500 CAPITOL,REDFORD,NaN,48239-2446,04/16/2021,...,NaN,NaN,Limited Liability Company(LLC),9551.0,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN
4,9609017706,05/01/2020,464.0,PPP,"D2 POWER SPORTS, LLC",125 Simuel Dr.,Spartanburg,NaN,29303-2085,NaN,...,NaN,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN
7,6486007709,05/01/2020,914.0,PPP,SEAN T KY DDS INC,200 S EL MOLINO AVE STE 5,PASADENA,NaN,91101-2985,NaN,...,NaN,NaN,Corporation,9551.0,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,NaN


In [89]:
# Check that lengths of DF split matches up
len(ppp_sub_150_1) == len(not_nan_df) + len(nan_zip_df)

True

### Part 2c. Add Modified Zip Code Column

Extract first 5 digits of `BorrowerZip` column

In [94]:
# Source: https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
pattern = '(^\d{5})'
not_nan_df['zip_code_5'] = not_nan_df['BorrowerZip'].str.extract(pattern)
not_nan_df.head()

C:\Users\Trang\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,zip_code_5
1,9677497701,05/01/2020,464.0,PPP,NORTH CHARLESTON HOSPITALITY GROUP LLC,192 College Park Rd,Ladson,NaN,29456-3517,NaN,...,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN,29456
2,9547167709,05/01/2020,464.0,PPP,Q AND J SERVICES LLC,301 Old Georgetown Road,Manning,NaN,29102-2734,04/20/2021,...,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN,29102
3,6448037706,05/01/2020,515.0,PPP,OPTIMIZED PROCESS SOLUTIONS DBA AAA INDUSTRIES,24500 CAPITOL,REDFORD,NaN,48239-2446,04/16/2021,...,NaN,Limited Liability Company(LLC),9551.0,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN,48239
4,9609017706,05/01/2020,464.0,PPP,"D2 POWER SPORTS, LLC",125 Simuel Dr.,Spartanburg,NaN,29303-2085,NaN,...,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN,29303
7,6486007709,05/01/2020,914.0,PPP,SEAN T KY DDS INC,200 S EL MOLINO AVE STE 5,PASADENA,NaN,91101-2985,NaN,...,NaN,Corporation,9551.0,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,NaN,91101


In [84]:
bay_area_ppp = not_nan_df[not_nan_df['zip_code_5'].isin(bay_area_zip['zip_code_5'])]
bay_area_ppp.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,zip_code_5
102272,8585717704,05/01/2020,459.0,PPP,MARKET AUTO TRUCK COLLISION CORPORATION,140 SAN JOSE AVE,SAN JOSE,AL,95125,NaN,...,NaN,Corporation,122043.0,WebBank,SALT LAKE CITY,UT,Unanswered,Unanswered,NaN,95125
392350,1464468601,03/13/2021,912.0,PPS,GOLD RUSH KETTLE KORN LLC,4690 E 2nd St Ste 9,Benicia,CA,94510-1008,NaN,...,NaN,Limited Liability Company(LLC),11822.0,"MUFG Union Bank, National Association",SAN FRANCISCO,CA,Male Owned,Non-Veteran,NaN,94510
392359,2663308509,02/22/2021,912.0,PPS,KALIBER LABS INC,188 King St Unit 307,San Francisco,CA,94107-4903,NaN,...,NaN,Corporation,9551.0,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN,94107
392364,4137348607,03/18/2021,912.0,PPS,ROSS MCDONALD COMPANY INC,1154 Stealth St,Livermore,CA,94551-9300,NaN,...,NaN,Corporation,474333.0,First Republic Bank,SAN FRANCISCO,CA,Unanswered,Unanswered,NaN,94551
392365,4315748602,03/18/2021,912.0,PPS,GATES EISENHART DAWSON,125 S Market St Ste 1200,San Jose,CA,95113-2288,NaN,...,NaN,Partnership,474333.0,First Republic Bank,SAN FRANCISCO,CA,Unanswered,Non-Veteran,NaN,95113


In [96]:
#Create new column for year of approval 
bay_area_ppp.loc[:,'YearApproved'] = bay_area_ppp['DateApproved'].str[-4:]

#Create new column for NAICS 6 digit in string format 
bay_area_ppp.loc[:,'NAICS_6'] = bay_area_ppp['NAICSCode'].apply(lambda y: str(y)[:6])

### Part 2d. Add `Minority` Column 

Options: Yes, No, Unanswered

In [118]:
bay_area_ppp[(bay_area_ppp['Ethnicity'] == 'Hispanic or Latino')]['Race']

# Question: Hispanic or Latino +  White? => Yes or No

392397                        White
392410                        White
392530                        White
392858                        White
392868                        White
                    ...            
899256                   Unanswered
899439                   Unanswered
899733    Black or African American
899775                        White
899949                        White
Name: Race, Length: 3274, dtype: object

In [120]:
# Source: https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/

# Question: does this cover all combos?
conditions = [
    (bay_area_ppp['Race']  == 'White'),
    (bay_area_ppp['Race'] == 'Unanswered') & (bay_area_ppp['Ethnicity'] == 'Not Hispanic or Latino'),
    (bay_area_ppp['Race'] == 'Unanswered') & (bay_area_ppp['Ethnicity'] == 'Unknown/NotStated'),
    (bay_area_ppp['Ethnicity'] == 'Hispanic or Latino') ,
    (bay_area_ppp['Race']  != 'White') & (bay_area_ppp['Race'] != 'Unanswered') ,
    ]

values = ['No', 'No', 'Unanswered', 'Yes', 'Yes']

# Note: 0 in Minority column indicates a condition that is not covered
bay_area_ppp['Minority'] = np.select(conditions, values)
bay_area_ppp.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,zip_code_5,YearApproved,NAICS_6,Minority
102272,8585717704,05/01/2020,459.0,PPP,MARKET AUTO TRUCK COLLISION CORPORATION,140 SAN JOSE AVE,SAN JOSE,AL,95125,NaN,...,WebBank,SALT LAKE CITY,UT,Unanswered,Unanswered,NaN,95125,2020,811121,Unanswered
392350,1464468601,03/13/2021,912.0,PPS,GOLD RUSH KETTLE KORN LLC,4690 E 2nd St Ste 9,Benicia,CA,94510-1008,NaN,...,"MUFG Union Bank, National Association",SAN FRANCISCO,CA,Male Owned,Non-Veteran,NaN,94510,2021,722330,No
392359,2663308509,02/22/2021,912.0,PPS,KALIBER LABS INC,188 King St Unit 307,San Francisco,CA,94107-4903,NaN,...,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN,94107,2021,541511,No
392364,4137348607,03/18/2021,912.0,PPS,ROSS MCDONALD COMPANY INC,1154 Stealth St,Livermore,CA,94551-9300,NaN,...,First Republic Bank,SAN FRANCISCO,CA,Unanswered,Unanswered,NaN,94551,2021,541420,Unanswered
392365,4315748602,03/18/2021,912.0,PPS,GATES EISENHART DAWSON,125 S Market St Ste 1200,San Jose,CA,95113-2288,NaN,...,First Republic Bank,SAN FRANCISCO,CA,Unanswered,Non-Veteran,NaN,95113,2021,541110,Unanswered


### Part 2e. Add NAICS 4-digit Column

In [126]:
#Merge NAICS 4 digit with PPP data and remove the 4 digit code from IndustrySubsector name
bay_area_ppp_NAICS = pd.merge(bay_area_ppp, NAICS_codes, how='left', left_on='NAICS_6', right_on='2017NAICSCode')
bay_area_ppp_NAICS.loc[:,'NAICS_4'] = bay_area_ppp_NAICS['4DigitNAICS'].apply(lambda y: str(y)[:4])
bay_area_ppp_NAICS = bay_area_ppp_NAICS.drop(['2017NAICSCode','Changed2017','TRIIndustrySector','NewIndustry','4DigitNAICS'], axis=1)
bay_area_ppp_NAICS['IndustrySubsector'] = bay_area_ppp_NAICS['IndustrySubsector'].str[5:]
bay_area_ppp_NAICS.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,Gender,Veteran,NonProfit,zip_code_5,YearApproved,NAICS_6,Minority,LongName,IndustrySubsector,NAICS_4
0,8585717704,05/01/2020,459.0,PPP,MARKET AUTO TRUCK COLLISION CORPORATION,140 SAN JOSE AVE,SAN JOSE,AL,95125,NaN,...,Unanswered,Unanswered,NaN,95125,2020,811121,Unanswered,"Automotive Body, Paint, and Interior Repair an...",Automotive Repair and Maintenance,8111
1,1464468601,03/13/2021,912.0,PPS,GOLD RUSH KETTLE KORN LLC,4690 E 2nd St Ste 9,Benicia,CA,94510-1008,NaN,...,Male Owned,Non-Veteran,NaN,94510,2021,722330,No,Mobile Food Services,Special Food Services,7223
2,2663308509,02/22/2021,912.0,PPS,KALIBER LABS INC,188 King St Unit 307,San Francisco,CA,94107-4903,NaN,...,Male Owned,Non-Veteran,NaN,94107,2021,541511,No,Custom Computer Programming Services,Computer Systems Design and Related Services,5415
3,4137348607,03/18/2021,912.0,PPS,ROSS MCDONALD COMPANY INC,1154 Stealth St,Livermore,CA,94551-9300,NaN,...,Unanswered,Unanswered,NaN,94551,2021,541420,Unanswered,Industrial Design Services,Specialized Design Services,5414
4,4315748602,03/18/2021,912.0,PPS,GATES EISENHART DAWSON,125 S Market St Ste 1200,San Jose,CA,95113-2288,NaN,...,Unanswered,Non-Veteran,NaN,95113,2021,541110,Unanswered,Offices of Lawyers,Legal Services,5411


## Part 3. Go through all files to build `PPP_combined`

In [50]:
def process_files():
    filenames = ['public_up_to_150k_{}.csv'.format(i) for i in range(1,12)]
    filenames.append('public_150k_plus.csv')
    nan_df_list = list()
    PPP_combined_list = list()
    for file in filenames:
        print('Processing {}...'.format(file))
        df = pd.read_csv(file)
                       
        nan_zip_df, not_nan_df = split_df_on_zip(df)
        
        # append nan zips from dataframe
        nan_df_list.append(nan_zip_df)
        
        # Add Zip Code, Minority, and NAICS_4 columns
        PPP_bay = add_zip_col(not_nan_df)
        PPP_bay = add_minority_col(PPP_bay) 
        PPP_bay = add_naics_4_col(PPP_bay)

        # append ppp_combined
        PPP_combined_list.append(PPP_bay)
    
    print('Finished reading files.')
    # Join dataframes across all files
    print('Joining dataframes...')
    main_nan_zip = pd.concat(nan_df_list)
    PPP_BA_combined = pd.concat(PPP_combined_list)
    
    
    # Save to files
    print('Saving to file...')
    main_nan_zip.to_csv('nan_zips.csv')
    PPP_BA_combined.to_csv('BA_bus_from_ppp.csv')
    
    print('done.')
    return main_nan_zip, PPP_BA_combined

In [51]:
def split_df_on_zip(df):
    # Count NaN Zip Codes (BorrowerZip) in table
    print('There are {} NaN values in this table.'.format(df['BorrowerZip'].isna().sum()))
    
    # Get rows where BorrowerZip is NaN
    nan_zip_df = df[df['BorrowerZip'].isnull()]
    # Get rows where BorrowerZip is not NaN
    not_nan_df = df[df['BorrowerZip'].notnull()]

    # Check that lengths of DF split matches up
    print('Split length matches total:', len(df) == len(not_nan_df) + len(nan_zip_df))
    
    return nan_zip_df, not_nan_df

In [52]:
def add_zip_col(not_nan_df):
    # Source: https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
    pattern = '(^\d{5})'
    not_nan_df['zip_code_5'] = not_nan_df['BorrowerZip'].str.extract(pattern)
    bay_area_ppp = not_nan_df[not_nan_df['zip_code_5'].isin(bay_area_zip['zip_code_5'])]
    
    return bay_area_ppp

In [53]:
def add_minority_col(bay_area_ppp):
    # Source: https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/

    # Question: does this cover all combos?
    conditions = [
        (bay_area_ppp['Race']  == 'White'),
        (bay_area_ppp['Race'] == 'Unanswered') & (bay_area_ppp['Ethnicity'] == 'Not Hispanic or Latino'),
        (bay_area_ppp['Race'] == 'Unanswered') & (bay_area_ppp['Ethnicity'] == 'Unknown/NotStated'),
        (bay_area_ppp['Ethnicity'] == 'Hispanic or Latino') ,
        (bay_area_ppp['Race']  != 'White') & (bay_area_ppp['Race'] != 'Unanswered') ,
        ]

    values = ['No', 'No', 'Unanswered', 'Yes', 'Yes']

    # Note: 0 in Minority column indicates a condition that is not covered
    bay_area_ppp.loc[:,'Minority'] = np.select(conditions, values)
    return bay_area_ppp

In [54]:
def add_naics_4_col(bay_area_ppp):
    #Create new column for year of approval 
    bay_area_ppp.loc[:,'YearApproved'] = bay_area_ppp['DateApproved'].str[-4:]

    #Create new column for NAICS 6 digit in string format 
    bay_area_ppp.loc[:,'NAICS_6'] = bay_area_ppp['NAICSCode'].apply(lambda y: str(y)[:6])
    
    #Merge NAICS 4 digit with PPP data and remove the 4 digit code from IndustrySubsector name
    bay_area_ppp_NAICS = pd.merge(bay_area_ppp, NAICS_codes, how='left', left_on='NAICS_6', right_on='2017NAICSCode')
    bay_area_ppp_NAICS.loc[:,'NAICS_4'] = bay_area_ppp_NAICS['4DigitNAICS'].apply(lambda y: str(y)[:4])
    bay_area_ppp_NAICS = bay_area_ppp_NAICS.drop(['2017NAICSCode','Changed2017','TRIIndustrySector','NewIndustry','4DigitNAICS'], axis=1)
    bay_area_ppp_NAICS.loc[:,'IndustrySubsector'] = bay_area_ppp_NAICS['IndustrySubsector'].str[5:]
    
    return bay_area_ppp_NAICS

In [55]:
nan_zips, PPP_combined = process_files()

Processing public_up_to_150k_1.csv...
There are 124 NaN values in this table.
Split length matches total: True
Processing public_up_to_150k_2.csv...
There are 8 NaN values in this table.
Split length matches total: True
Processing public_up_to_150k_3.csv...
There are 3 NaN values in this table.
Split length matches total: True
Processing public_up_to_150k_4.csv...
There are 4 NaN values in this table.
Split length matches total: True
Processing public_up_to_150k_5.csv...
There are 4 NaN values in this table.
Split length matches total: True
Processing public_up_to_150k_6.csv...
There are 0 NaN values in this table.
Split length matches total: True
Processing public_up_to_150k_7.csv...
There are 2 NaN values in this table.
Split length matches total: True
Processing public_up_to_150k_8.csv...
There are 7 NaN values in this table.
Split length matches total: True
Processing public_up_to_150k_9.csv...
There are 0 NaN values in this table.
Split length matches total: True
Processing public

In [56]:
nan_zips.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
0,5375617707,05/01/2020,101.0,PPP,NOT AVAILABLE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9551.0,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,NaN
5,9789867710,05/01/2020,101.0,PPP,VULCAN MACHINERY CORPORATION,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Corporation,57328.0,The Huntington National Bank,COLUMBUS,OH,Male Owned,Non-Veteran,NaN
6,9589997709,05/01/2020,101.0,PPP,"TJK KITCHENS &AMP; BREWPUBS, LLC",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Limited Liability Company(LLC),57328.0,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN
8,9662387700,05/01/2020,101.0,PPP,RON GOLDSTONE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,57328.0,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN
9,2767027201,04/16/2020,NaN,PPP,Exemption 6,NaN,NaN,NaN,NaN,02/18/2021,...,0.0,0.0,Subchapter S Corporation,NaN,NaN,NaN,NaN,Unanswered,Unanswered,NaN


In [57]:
PPP_combined.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,Gender,Veteran,NonProfit,zip_code_5,Minority,YearApproved,NAICS_6,LongName,IndustrySubsector,NAICS_4
0,8585717704,05/01/2020,459.0,PPP,MARKET AUTO TRUCK COLLISION CORPORATION,140 SAN JOSE AVE,SAN JOSE,AL,95125,NaN,...,Unanswered,Unanswered,NaN,95125,Unanswered,2020,811121,"Automotive Body, Paint, and Interior Repair an...",Automotive Repair and Maintenance,8111
1,1464468601,03/13/2021,912.0,PPS,GOLD RUSH KETTLE KORN LLC,4690 E 2nd St Ste 9,Benicia,CA,94510-1008,NaN,...,Male Owned,Non-Veteran,NaN,94510,No,2021,722330,Mobile Food Services,Special Food Services,7223
2,2663308509,02/22/2021,912.0,PPS,KALIBER LABS INC,188 King St Unit 307,San Francisco,CA,94107-4903,NaN,...,Male Owned,Non-Veteran,NaN,94107,No,2021,541511,Custom Computer Programming Services,Computer Systems Design and Related Services,5415
3,4137348607,03/18/2021,912.0,PPS,ROSS MCDONALD COMPANY INC,1154 Stealth St,Livermore,CA,94551-9300,NaN,...,Unanswered,Unanswered,NaN,94551,Unanswered,2021,541420,Industrial Design Services,Specialized Design Services,5414
4,4315748602,03/18/2021,912.0,PPS,GATES EISENHART DAWSON,125 S Market St Ste 1200,San Jose,CA,95113-2288,NaN,...,Unanswered,Non-Veteran,NaN,95113,Unanswered,2021,541110,Offices of Lawyers,Legal Services,5411


In [59]:
def extract_bay_minority_bus(df):
    BA_minority_PPP = df[df['Minority'] == 'Yes']
    BA_minority_PPP.to_csv('BA_minority_bus_from_PPP.csv')
    return BA_minority_PPP
extract_bay_minority_bus(PPP_combined)

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,Gender,Veteran,NonProfit,zip_code_5,Minority,YearApproved,NAICS_6,LongName,IndustrySubsector,NAICS_4
6,5224738402,02/08/2021,912.0,PPS,ICHINA,70 Valley Ct,Atherton,CA,94027-6472,NaN,...,Female Owned,Non-Veteran,NaN,94027,Yes,2021,722511,Full-Service Restaurants,Restaurants and Other Eating Places,7225
7,5359188509,02/27/2021,912.0,PPS,BANZAI IZAKAYA INC.,1633 Bonanza St,Walnut Creek,CA,94596-4525,NaN,...,Male Owned,Non-Veteran,NaN,94596,Yes,2021,722511,Full-Service Restaurants,Restaurants and Other Eating Places,7225
9,5554978603,03/20/2021,912.0,PPS,RANGOON SUPER STARS LLC,542 Freya Way,Pleasant Hill,CA,94523-1712,NaN,...,Male Owned,Non-Veteran,NaN,94523,Yes,2021,722511,Full-Service Restaurants,Restaurants and Other Eating Places,7225
17,9634738305,01/31/2021,912.0,PPS,JBR PARTNERS INC,1333 Evans Ave,San Francisco,CA,94124-1705,NaN,...,Male Owned,Non-Veteran,NaN,94124,Yes,2021,541820,Public Relations Agencies,"Advertising, Public Relations, and Related Ser...",5418
18,5004708402,02/07/2021,912.0,PPS,OSAKE INC,2446 Patio Ct,Santa Rosa,CA,95405-6737,NaN,...,Female Owned,Non-Veteran,NaN,95405,Yes,2021,722511,Full-Service Restaurants,Restaurants and Other Eating Places,7225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33133,8589708302,01/29/2021,912.0,PPS,EMBODYVR INC.,60 E 3rd Ave Ste 120,San Mateo,CA,94401-4034,NaN,...,Male Owned,Non-Veteran,NaN,94401,Yes,2021,511210,Software Publishers,Software Publishers,5112
33134,8664418408,02/13/2021,912.0,PPS,OTORRES LANDSCAPING INC,10691 La Honda Rd,Woodside,CA,94062-5700,NaN,...,Male Owned,Non-Veteran,NaN,94062,Yes,2021,561730,Landscaping Services,Services to Buildings and Dwellings,5617
33145,9120708506,03/12/2021,912.0,PPS,A TOUCH OF ELEGANCE INC.,1150 Crane St,Menlo Park,CA,94025-4314,NaN,...,Female Owned,Non-Veteran,NaN,94025,Yes,2021,812112,Beauty Salons,Personal Care Services,8121
33150,9360078408,02/16/2021,912.0,PPS,SIRI GROUP INC.,1175 Folsom St,San Francisco,CA,94103-3930,NaN,...,Female Owned,Non-Veteran,NaN,94103,Yes,2021,722511,Full-Service Restaurants,Restaurants and Other Eating Places,7225


## PPP Loan Data (Unfiltered)

By Laura Schmahmann

Note: Heavy on PC memory, couldn't run on my end, see above for filtered version.

In [3]:
#Import Paycheck Protection Program Loan Information
ppp_sub_150_1 = pd.read_csv('public_up_to_150k_1.csv')
ppp_sub_150_2 = pd.read_csv('public_up_to_150k_2.csv')
ppp_sub_150_3 = pd.read_csv('public_up_to_150k_3.csv')
ppp_sub_150_4 = pd.read_csv('public_up_to_150k_4.csv')
ppp_sub_150_5 = pd.read_csv('public_up_to_150k_5.csv')
ppp_sub_150_6 = pd.read_csv('public_up_to_150k_6.csv')
ppp_sub_150_7 = pd.read_csv('public_up_to_150k_7.csv')
ppp_sub_150_8 = pd.read_csv('public_up_to_150k_8.csv')
ppp_sub_150_9 = pd.read_csv('public_up_to_150k_9.csv')
ppp_sub_150_10 = pd.read_csv('public_up_to_150k_10.csv')
ppp_sub_150_11 = pd.read_csv('public_up_to_150k_11.csv')
ppp_plus_150 = pd.read_csv('public_150k_plus.csv')


C:\Users\Trang\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (16,50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Trang\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
ppp_plus_150.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
0,9547507704,05/01/2020,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,...,NaN,NaN,Corporation,19248,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN
1,9777677704,05/01/2020,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,NaN,...,NaN,NaN,Sole Proprietorship,19248,Synovus Bank,COLUMBUS,GA,Male Owned,Non-Veteran,NaN
2,5791407702,05/01/2020,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,...,NaN,NaN,Non-Profit Organization,9551,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,Y
3,6223567700,05/01/2020,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,NaN,...,NaN,NaN,Corporation,9551,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,NaN
4,9662437702,05/01/2020,101,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,57328,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN


In [5]:
#Combine PPP datasets
PPP_datasets = [ppp_sub_150_1,ppp_sub_150_2,ppp_sub_150_3,ppp_sub_150_4,ppp_sub_150_5,ppp_sub_150_6,
                ppp_sub_150_7,ppp_sub_150_8,ppp_sub_150_9,ppp_sub_150_10,ppp_sub_150_11,ppp_plus_150]

In [6]:
PPP_combined = pd.concat(PPP_datasets)

In [7]:
PPP_combined.head(10)

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
0,5375617707,05/01/2020,101.0,PPP,NOT AVAILABLE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9551.0,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,NaN
1,9677497701,05/01/2020,464.0,PPP,NORTH CHARLESTON HOSPITALITY GROUP LLC,192 College Park Rd,Ladson,NaN,29456-3517,NaN,...,NaN,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN
2,9547167709,05/01/2020,464.0,PPP,Q AND J SERVICES LLC,301 Old Georgetown Road,Manning,NaN,29102-2734,04/20/2021,...,NaN,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN
3,6448037706,05/01/2020,515.0,PPP,OPTIMIZED PROCESS SOLUTIONS DBA AAA INDUSTRIES,24500 CAPITOL,REDFORD,NaN,48239-2446,04/16/2021,...,NaN,NaN,Limited Liability Company(LLC),9551.0,"Bank of America, National Association",CHARLOTTE,NC,Male Owned,Non-Veteran,NaN
4,9609017706,05/01/2020,464.0,PPP,"D2 POWER SPORTS, LLC",125 Simuel Dr.,Spartanburg,NaN,29303-2085,NaN,...,NaN,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN
5,9789867710,05/01/2020,101.0,PPP,VULCAN MACHINERY CORPORATION,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Corporation,57328.0,The Huntington National Bank,COLUMBUS,OH,Male Owned,Non-Veteran,NaN
6,9589997709,05/01/2020,101.0,PPP,"TJK KITCHENS &AMP; BREWPUBS, LLC",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Limited Liability Company(LLC),57328.0,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN
7,6486007709,05/01/2020,914.0,PPP,SEAN T KY DDS INC,200 S EL MOLINO AVE STE 5,PASADENA,NaN,91101-2985,NaN,...,NaN,NaN,Corporation,9551.0,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,NaN
8,9662387700,05/01/2020,101.0,PPP,RON GOLDSTONE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,57328.0,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN
9,2767027201,04/16/2020,NaN,PPP,Exemption 6,NaN,NaN,NaN,NaN,02/18/2021,...,0.0,0.0,Subchapter S Corporation,NaN,NaN,NaN,NaN,Unanswered,Unanswered,NaN


In [8]:
PPP_combined.columns

Index(['LoanNumber', 'DateApproved', 'SBAOfficeCode', 'ProcessingMethod',
       'BorrowerName', 'BorrowerAddress', 'BorrowerCity', 'BorrowerState',
       'BorrowerZip', 'LoanStatusDate', 'LoanStatus', 'Term',
       'SBAGuarantyPercentage', 'InitialApprovalAmount',
       'CurrentApprovalAmount', 'UndisbursedAmount', 'FranchiseName',
       'ServicingLenderLocationID', 'ServicingLenderName',
       'ServicingLenderAddress', 'ServicingLenderCity', 'ServicingLenderState',
       'ServicingLenderZip', 'RuralUrbanIndicator', 'HubzoneIndicator',
       'LMIIndicator', 'BusinessAgeDescription', 'ProjectCity',
       'ProjectCountyName', 'ProjectState', 'ProjectZip', 'CD', 'JobsReported',
       'NAICSCode', 'Race', 'Ethnicity', 'UTILITIES_PROCEED',
       'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED',
       'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED',
       'DEBT_INTEREST_PROCEED', 'BusinessType', 'OriginatingLenderLocationID',
       'OriginatingLender', 'Originatin

In [9]:
PPP_combined['Race'].unique()

array(['Unanswered', 'White', 'Black or African American', 'Asian',
       'American Indian or Alaska Native',
       'Native Hawaiian or Other Pacific Islander', 'Eskimo & Aleut',
       'Multi Group', 'Puerto Rican'], dtype=object)

In [10]:
PPP_combined['Ethnicity'].unique()

array(['Unknown/NotStated', 'Not Hispanic or Latino',
       'Hispanic or Latino'], dtype=object)

In [11]:
#Create new column for year of approval 
PPP_combined['YearApproved'] = PPP_combined['DateApproved'].str[-4:]

#Create new column for NAICS 6 digit in string format 
PPP_combined['NAICS_6'] = PPP_combined['NAICSCode'].apply(lambda y: str(y)[:6])

In [13]:
#Create new column identifying minority-owned businesses as identified by Race and Ethnicity columns
PPP_combined.loc[PPP_combined['Race'] == 'American Indian or Alaska Native', 'Minority'] = 'Yes'  
PPP_combined.loc[PPP_combined['Race'] == 'Asian', 'Minority'] = 'Yes'  
PPP_combined.loc[PPP_combined['Race'] == 'Black or African American', 'Minority'] = 'Yes'  
PPP_combined.loc[PPP_combined['Race'] == 'Hispanic', 'Minority'] = 'Yes'  
PPP_combined.loc[PPP_combined['Race'] == 'Multi Group', 'Minority'] = 'Yes'  
PPP_combined.loc[PPP_combined['Race'] == 'Puerto Rican', 'Minority'] = 'Yes' 
PPP_combined.loc[PPP_combined['Race'] == 'Native Hawaiian or Other Pacific Islander', 'Minority'] = 'Yes' 
PPP_combined.loc[PPP_combined['Race'] == 'Eskimo & Aleut', 'Minority'] = 'Yes' 
PPP_combined.loc[PPP_combined['Race'] == 'White', 'Minority'] = 'No'
PPP_combined.loc[PPP_combined['Race'] == 'Unanswered', 'Minority'] = 'Unanswered'
PPP_combined.loc[PPP_combined['Ethnicity'] == 'Hispanic or Latino', 'Minority'] = 'Yes'

In [ ]:
#Merge NAICS 4 digit with PPP data and remove the 4 digit code from IndustrySubsector name
PPP_combined_NAICS = pd.merge(PPP_combined, NAICS_codes, how='left', left_on='NAICS_6', right_on='2017NAICSCode')
PPP_combined_NAICS.loc[:,'NAICS_4'] = PPP_combined_NAICS['4DigitNAICS'].apply(lambda y: str(y)[:4])
PPP_combined_NAICS = PPP_combined_NAICS.drop(['2017NAICSCode','Changed2017','TRIIndustrySector','NewIndustry','4DigitNAICS'], axis=1)
PPP_combined_NAICS['IndustrySubsector'] = PPP_combined_NAICS['IndustrySubsector'].str[5:]
PPP_combined_NAICS.head()

In [ ]:
PPP_combined_NAICS['BorrowerState'].unique()

In [ ]:
PPP_combined_NAICS_CA = PPP_combined_NAICS[PPP_combined_NAICS['BorrowerState']=='CA']
PPP_combined_NAICS_CA.head()

In [ ]:
PPP_combined_NAICS_no_state = PPP_combined_NAICS[PPP_combined_NAICS['BorrowerState'].isna()]
PPP_combined_NAICS_no_state.head()

In [ ]:
PPP_combined_NAICS_CA_2020 = PPP_combined_NAICS_CA[PPP_combined_NAICS_CA['YearApproved']=='2020']
PPP_combined_NAICS_CA_2021 = PPP_combined_NAICS_CA[PPP_combined_NAICS_CA['YearApproved']=='2021']

In [ ]:
#Write to CSV
PPP_combined_NAICS_CA_2020.to_csv('PPP_extract_CA_20.csv')
PPP_combined_NAICS_CA_2021.to_csv('PPP_extract_CA_21.csv')

In [ ]:
#Write to CSV
PPP_combined_NAICS_no_state.to_csv('PPP_extract_no_state.csv')

## Summary Data Tables 

In [ ]:
PPP_ind_4 = pd.DataFrame(PPP_combined_NAICS.groupby(['NAICS_4','IndustrySubsector'])['JobsReported']
                          .agg(Businesses='count',Jobs='sum').sort_values(by='Businesses', ascending=False))
PPP_ind_4.reset_index(inplace=True)
PPP_ind_4.index = np.arange(1, len(PPP_ind_4) + 1)
PPP_ind_4.head(20)

In [ ]:
retail_4 = ['4411', '4412', '4413', '4421', '4422', '4431', '4441', '4442', '4451', '4452', '4453', '4461', '4471', '4481', 
            '4482', '4483', '4511', '4512', '4522', '4523', '4531', '4532', '4533', '4539', '4541', '4542', '4543']

In [ ]:
PPP_combined_NAICS['NAICS_4'].dtypes

In [ ]:
PPP_retail_4 = PPP_ind_4[PPP_ind_4['NAICS_4'].isin(retail_4)]
PPP_retail_4

In [ ]:
personal_4 = ['8121', '8122', '8123', '8129']

In [ ]:
PPP_personal_4 = PPP_ind_4[PPP_ind_4['NAICS_4'].isin(personal_4)]
PPP_personal_4

In [ ]:
food_4 = ['7223', '7224', '7225']

In [ ]:
PPP_food_4 = PPP_ind_4[PPP_ind_4['NAICS_4'].isin(food_4)]
PPP_food_4

In [ ]:
PPP_combined_minority = PPP_combined_NAICS[PPP_combined_NAICS['Minority']=='Yes']
PPP_combined_minority.head(10)

In [ ]:
PPP_minority_ind_4 = pd.DataFrame(PPP_combined_minority.groupby(['NAICS_4','IndustrySubsector'])['JobsReported']
                          .agg(Businesses='count',Jobs='sum').sort_values(by='Businesses', ascending=False))
PPP_minority_ind_4.reset_index(inplace=True)
PPP_minority_ind_4.index = np.arange(1, len(PPP_minority_ind_4) + 1)
PPP_minority_ind_4.head(20)

In [ ]:
PPP_minority_retail_4 = PPP_minority_ind_4[PPP_minority_ind_4['NAICS_4'].isin(retail_4)]
PPP_minority_retail_4

In [ ]:
PPP_combined_NAICS['YearApproved'].unique()

In [ ]:
#Extract for 2021 approvals only 
PPP_combined_2021 = PPP_combined_NAICS[PPP_combined_NAICS['YearApproved']=='2021']
PPP_combined_2021.head(10)

In [ ]:
PPP_ind_4_2021 = pd.DataFrame(PPP_combined_2021.groupby(['NAICS_4','IndustrySubsector'])['JobsReported']
                          .agg(Businesses='count',Jobs='sum').sort_values(by='Businesses', ascending=False))
PPP_ind_4_2021.reset_index(inplace=True)
PPP_ind_4_2021.index = np.arange(1, len(PPP_top_ind) + 1)
PPP_ind_4_2021.head(20)

In [ ]:
PPP_retail_4_2021 = PPP_ind_4_2021[PPP_ind_4_2021['NAICS_4'].isin(retail_4)]
PPP_retail_4_2021

In [ ]:
PPP_combined_minority_2021 = PPP_combined_2021[PPP_combined_2021['Minority']=='Yes']
PPP_combined_minority_2021.head(10)

In [ ]:
PPP_minority_ind_4_2021 = pd.DataFrame(PPP_combined_minority_2021.groupby(['NAICS_4','IndustrySubsector'])['JobsReported']
                          .agg(Businesses='count',Jobs='sum').sort_values(by='Businesses', ascending=False))
PPP_minority_ind_4_2021.reset_index(inplace=True)
PPP_minority_ind_4_2021.index = np.arange(1, len(PPP_minority_ind_4_2021) + 1)
PPP_minority_ind_4_2021.head(20)

In [ ]:
PPP_minority_retail_4_2021 = PPP_minority_ind_4_2021[PPP_minority_ind_4_2021['NAICS_4'].isin(retail_4)]
PPP_minority_retail_4_2021